## XGBoost-Optunaのサンプルコード

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

# Optuna
import optuna

# XGBoost
import xgboost as xgb
from optuna.samplers import CmaEsSampler

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 評価指標
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split

# 前処理
from sklearn.preprocessing import StandardScaler

In [3]:
print(xgb.__version__)

1.3.3


In [4]:
print(optuna.__version__)

2.8.0


In [5]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [6]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

In [7]:
def objective(trial):

    eta = trial.suggest_loguniform("eta", 1e-8, 1.0)
    gamma = trial.suggest_loguniform("gamma", 1e-8, 1.0)
    max_depth = trial.suggest_int("max_depth", 3, 8)
    min_child_weight = trial.suggest_loguniform("min_child_weight", 1, 40)
    max_delta_step = trial.suggest_loguniform("max_delta_step", 1e-8, 1.0)
    subsample = trial.suggest_uniform("subsample", 0.0, 1.0)
    reg_lambda = trial.suggest_uniform("reg_lambda", 0.0, 1000.0)
    reg_alpha = trial.suggest_uniform("reg_alpha", 0.0, 1000.0)

    model = xgb.XGBRegressor(
        eta=eta,
        gamma=gamma,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        max_delta_step=max_delta_step,
        subsample=subsample,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
    )

    score = cross_val_score(
        model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error"
    )
    mae = score.mean()

    return mae

In [8]:
%%time
# optunaで最適値を見つける
# 注：cross_val_scoreの出力は全て高いほど良い
study = optuna.create_study(
    direction="maximize", sampler=CmaEsSampler(seed=RANDOM_STATE)
)
study.optimize(objective, n_trials=500)

[I 2021-07-08 06:35:18,305] A new study created in memory with name: no-name-8c57dcf3-3426-431e-b269-9bd85dd36946
[I 2021-07-08 06:35:18,478] Trial 0 finished with value: -21.343721146383405 and parameters: {'eta': 0.014810344004555135, 'gamma': 1.4656004675652718e-08, 'max_depth': 6, 'min_child_weight': 15.83538918896983, 'max_delta_step': 9.728728830009641e-05, 'subsample': 0.22479664553084766, 'reg_lambda': 198.06286475962398, 'reg_alpha': 760.5307121989588}. Best is trial 0 with value: -21.343721146383405.
[I 2021-07-08 06:35:18,603] Trial 1 finished with value: -21.343864197530866 and parameters: {'eta': 0.00010395554367766142, 'gamma': 0.00011981534473045809, 'max_depth': 5, 'min_child_weight': 5.268343923842643, 'max_delta_step': 9.141971109173533e-05, 'subsample': 0.6354789588989669, 'reg_lambda': 499.983711501389, 'reg_alpha': 499.6568158136499}. Best is trial 0 with value: -21.343721146383405.
[I 2021-07-08 06:35:18,704] Trial 2 finished with value: -21.343864197530866 and pa

CPU times: user 4min 52s, sys: 16.4 s, total: 5min 8s
Wall time: 1min 30s


In [9]:
# チューニングしたハイパーパラメータ
study.best_params

{'eta': 0.014810344004555135,
 'gamma': 1.4656004675652718e-08,
 'max_depth': 6,
 'min_child_weight': 15.83538918896983,
 'max_delta_step': 9.728728830009641e-05,
 'subsample': 0.22479664553084766,
 'reg_lambda': 198.06286475962398,
 'reg_alpha': 760.5307121989588}

In [10]:
# チューニングしたハイパーパラメーターをフィット
optimised_model = xgb.XGBRegressor(
    eta=study.best_params["eta"],
    gamma=study.best_params["gamma"],
    max_depth=study.best_params["max_depth"],
    min_child_weight=study.best_params["min_child_weight"],
    max_delta_step=study.best_params["max_delta_step"],
    subsample=study.best_params["subsample"],
    reg_lambda=study.best_params["reg_lambda"],
    reg_alpha=study.best_params["reg_alpha"],
)

optimised_model.fit(x_train, y_train)

# XGBoost推論
y_pred = optimised_model.predict(x_test)

In [11]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [12]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2        MAE         MSE       RMSE
scores -5.866533  24.769465  718.107129  26.797521
